# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Описание данных**

Признаки

DateCrawled — дата скачивания анкеты из базы

VehicleType — тип автомобильного кузова

RegistrationYear — год регистрации автомобиля

Gearbox — тип коробки передач

Power — мощность (л. с.)

Model — модель автомобиля

Kilometer — пробег (км)

RegistrationMonth — месяц регистрации автомобиля

FuelType — тип топлива

Brand — марка автомобиля

Repaired — была машина в ремонте или нет

DateCreated — дата создания анкеты

NumberOfPictures — количество фотографий автомобиля

PostalCode — почтовый индекс владельца анкеты (пользователя)

LastSeen — дата последней активности пользователя

Целевой признак

Price — цена (евро)

## Подготовка данных

In [1]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder

In [3]:
data = pd.read_csv('datasets/autos.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

У некоторых столбцов отсутствуют данные. Невозможно заменить их на основе других столбцов, и использование значения "неизвестно" может привести к искажению результатов. Поскольку пропущенные значения составляют небольшую долю от общего объема данных, можем удалить их.

In [7]:
data["Repaired"] = data["Repaired"].fillna("yes")
data['VehicleType'] = data['VehicleType'].fillna('Unknown')
data['Gearbox'] = data['Gearbox'].fillna('Unknown')
data['Model'] = data['Model'].fillna('Unknown')
data["FuelType"] = data["FuelType"].fillna('Unknown')

Приведем названия столбцов к нижнему регистру.

In [8]:
data.columns = data.columns.str.lower()

Проверим, присутствуют ли дубликаты, и если таковые обнаружатся, удалим их.

In [9]:
data.duplicated().sum()

4

In [10]:
data = data.drop_duplicates().reset_index(drop=True)

Исключим из анализа столбцы, которые не являются необходимыми для наших дальнейших задач, так как они не содержат значимой информации для нас.

In [11]:
data = data.drop(["datecrawled","datecreated","lastseen","numberofpictures","postalcode"],axis = 1)

In [12]:
data.head()

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
0,480,Unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,yes
1,18300,coupe,2011,manual,190,Unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,yes
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


Осуществим проверку на наличие аномалий.

In [13]:
data.price.unique()

array([  480, 18300,  9800, ..., 12395, 18429, 10985], dtype=int64)

In [14]:
data = data.query('price >= 50')

In [15]:
data.vehicletype.unique()

array(['Unknown', 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

In [16]:
data.registrationyear.unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 2014, 1998, 2005, 1910, 2016,
       2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994, 1991,
       1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985, 1989,
       2015, 1982, 1976, 1983, 1973, 1969, 1971, 1987, 1986, 1988, 1980,
       1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968, 1977,
       1961, 1966, 1975, 1963, 1964, 1960, 1958, 1967, 1956, 3200, 1000,
       1941, 9999, 8888, 1500, 2200, 4100, 1962, 1929, 1957, 1940, 3000,
       2066, 1949, 2019, 1937, 1951, 1800, 1959, 1953, 1954, 1234, 8000,
       5300, 9000, 2900, 6000, 5900, 5911, 1400, 5000, 1950, 4000, 1948,
       1952, 8500, 1932, 1255, 3700, 3800, 4800, 1942, 7000, 1935, 1933,
       1936, 1923, 2290, 1930, 1001, 9450, 1944, 2500, 1943, 1938, 1934,
       1928, 5555, 5600, 1600, 1111, 2222, 1039, 1300, 2800, 1931, 4500,
       1602, 7800, 1947, 1927, 7100, 8200, 1946], dtype=int64)

Обнаружена аномалия, исключим из данных значения, предшествующие 1900 году и превышающие 2016 год.

In [17]:
data = data.loc[(data['registrationyear'] <= 2016) & (data['registrationyear'] >= 1900)]

In [18]:
data.gearbox.unique()

array(['manual', 'auto', 'Unknown'], dtype=object)

In [19]:
data.power.unique()

array([    0,   190,   163,    75,    69,   102,   109,   125,   101,
         105,   140,   115,   131,    60,   136,   160,   231,    50,
         118,   193,    99,   113,   218,   122,   129,    70,   306,
          95,    61,   177,   170,    55,   143,   232,   150,   156,
          80,    82,    90,   155,    54,   185,    87,   180,    86,
          84,   224,   235,   200,   178,   265,    77,   110,   144,
         120,   286,   116,   184,   126,   204,    88,   194,    64,
         305,   197,   179,   250,    45,   313,    41,   165,    98,
         130,   114,   211,    56,   201,   213,    58,   107,    83,
         174,   100,   220,    73,   192,    68,    66,   299,    74,
          52,   147,   310,    71,    97,    65,   239,   203,     5,
         300,   103,    85,   258,   320,    63,    81,   148,    44,
         145,   280,   260,   104,   188,   333,   186,   117,   141,
         132,   234,   158,    39,    92,    51,   135,    59,   230,
          53,   209,

Обнаружена аномалия, исключим из данных значения мощности автомобиля, превышающие 700.

In [20]:
data = data.loc[data['power'] <= 700]

Исключим из данных значения можности автомобиля, равный 0

In [21]:
data = data.drop(data[data['power'] == 0].index)

In [22]:
data.model.unique()

array(['Unknown', 'grand', 'golf', 'fabia', '3er', '2_reihe', 'c_max',
       '3_reihe', 'passat', 'navara', 'polo', 'twingo', 'a_klasse',
       'scirocco', '5er', 'arosa', 'other', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kadett', 'one', 'fortwo', '1er',
       'b_klasse', 'a8', 'jetta', 'fiesta', 'c_klasse', 'micra', 'vito',
       'sprinter', 'astra', '156', 'escort', 'forester', 'xc_reihe',
       'scenic', 'ka', 'a1', 'focus', 'a4', 'tt', 'a6', 'jazz', 'omega',
       'slk', '7er', 'combo', 'corsa', '80', '147', 'z_reihe', 'sorento',
       'ibiza', 'mustang', 'eos', 'touran', 'getz', 'insignia', 'almera',
       'megane', 'a3', 'lupo', 'r19', 'caddy', 'mondeo', 'cordoba',
       'colt', 'impreza', 'vectra', 'berlingo', 'tiguan', '6_reihe', 'c4',
       'panda', 'up', 'i_reihe', 'ceed', 'kangoo', '5_reihe', 'yeti',
       'octavia', 'zafira', 'mii', 'rx_reihe', '6er', 'modus', 'fox',
       'matiz', 'beetle', 'rio', 'touareg', 'logan', 'spider', 'cuore',
     

In [23]:
data.kilometer.unique()

array([125000, 150000,  90000,  30000,  70000, 100000,  60000,   5000,
        20000,  80000,  50000,  40000,  10000], dtype=int64)

In [24]:
data.registrationmonth.unique()

array([ 5,  8,  6,  7, 10,  0, 12, 11,  2,  3,  1,  4,  9], dtype=int64)

In [25]:
data.fueltype.unique()

array(['gasoline', 'petrol', 'Unknown', 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

In [26]:
data.brand.unique()

array(['audi', 'jeep', 'volkswagen', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'seat', 'honda',
       'fiat', 'opel', 'mini', 'smart', 'hyundai', 'alfa_romeo', 'subaru',
       'volvo', 'mitsubishi', 'kia', 'suzuki', 'lancia', 'citroen',
       'toyota', 'chevrolet', 'dacia', 'sonstige_autos', 'daihatsu',
       'chrysler', 'jaguar', 'daewoo', 'rover', 'porsche', 'saab',
       'trabant', 'land_rover', 'lada'], dtype=object)

In [27]:
data.repaired.unique()

array(['yes', 'no'], dtype=object)

In [28]:
data.duplicated().sum()

23468

In [29]:
data = data.drop_duplicates()

In [30]:
data.duplicated().sum()

0

При анализе данных были обнаружены пропущенные значения, которые были удалены. Также были обнаружены и удалены дубликаты. Были исключены из данных столбцы, не содержащие значимой информации. Были обработаны аномалии в столбцах, относящихся к году регистрации и мощности автомобилей.

## Обучение моделей

In [31]:
features=data.drop(['price'],axis=1)
target=data['price']

In [32]:
col=['vehicletype', 'gearbox','model',
     'fueltype', 'brand', 'repaired']

Делим данные на выборки

In [33]:
features_train, features_valid_test, target_train, target_valid_test  = train_test_split(features,target, test_size=0.4, random_state=254)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

features_train.shape, features_valid.shape, features_test.shape

((163414, 10), (54472, 10), (54472, 10))

Копируем данные для кодировки

In [34]:
features_train_oe = features_train.copy()
features_valid_oe = features_valid.copy()
features_test_oe = features_test.copy()

In [35]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(features_train[col])
features_train_oe[col] = encoder.transform(features_train_oe[col])
features_valid_oe[col] = encoder.transform(features_valid_oe[col])
features_test_oe[col] = encoder.transform(features_test_oe[col])
features_test_oe.head()

,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
178404,6.0,2003,2.0,60,5.0,150000,6,7.0,25.0,0.0
139764,5.0,2011,2.0,105,117.0,125000,12,3.0,38.0,0.0
43021,8.0,2001,2.0,140,137.0,150000,10,7.0,27.0,1.0
200529,6.0,2004,2.0,98,76.0,150000,12,7.0,27.0,0.0
339321,8.0,2003,2.0,75,43.0,150000,3,3.0,24.0,1.0


Проведем обучение модели дерева принятия решений

In [36]:
%%time
depth_final = 0
rmse_final = 111110

for depth in range(3, 20, 3):
     model = DecisionTreeRegressor(random_state=12345, max_depth = depth)
     model.fit(features_train_oe, target_train)
     predictions = model.predict(features_valid_oe)
     rmse = mean_squared_error(target_valid, predictions)**0.5

     if rmse <  rmse_final :
          rmse_final = rmse
          depth_final = depth

print('Лучший результат: Глубина', depth_final, ', RMSE', rmse_final )

Лучший результат: Глубина 12 , RMSE 1923.7199361444189
Wall time: 5.39 s


Проведем обучение модели, использующей алгоритм случайного леса.

In [37]:
%%time
depth_final = 0
rmse_final = 111110
est_final = 0

for depth in range(1,5):
     for est in range(5,20,5):
          model = RandomForestRegressor(max_depth=depth, n_estimators=est, random_state=1234)
          model.fit(features_train_oe, target_train)
          predictions = model.predict(features_valid_oe)
          rmse = mean_squared_error(target_valid, predictions)**0.5

          if rmse <  rmse_final :
               rmse_final = rmse
               depth_final = depth
               est_final = est

print('Лучший результат: Количество деревьев', est_final, ", Глубина",depth_final, ', RMSE для случайного леса', rmse_final)

Лучший результат: Количество деревьев 5 , Глубина 4 , RMSE для случайного леса 2567.134303713294
Wall time: 16.5 s


Проведем обучение модели, использующей алгоритм LGBMRegressor

In [38]:
%%time
num_final = 0
rmse_final = 111110
est_final = 0

for est in [100, 500, 1000]:
     for num in [25, 400, 15]:
          model = lgb.LGBMRegressor(n_estimators = est, num_leaves = num, random_state=12345)
          model.fit(features_train_oe, target_train)
          predictions= model.predict(features_valid_oe)
          rmse = mean_squared_error(target_valid, predictions)**0.5

          if rmse <  rmse_final :
               rmse_final = rmse
               num_final = num
               est_final = est

print('Лучший результат: Параметры n_estimators',  est_final,', num_leaves', num_final, ', RMSE для LightGBM', rmse_final)


Лучший результат: Параметры n_estimators 1000 , num_leaves 25 , RMSE для LightGBM 1576.881736930125
Wall time: 1min 3s


Среди трех обученных моделей - решающего дерева, случайного леса и LightGBM - модель LightGBM показала наилучший результат по метрике RMSE, достигнув значения 1576.881736930125.

## Анализ моделей

In [39]:
%%time
model1 = DecisionTreeRegressor(random_state=12345, max_depth=12)
model1.fit(features_train_oe, target_train)

Wall time: 490 ms


DecisionTreeRegressor(max_depth=12, random_state=12345)

In [40]:
%%time
predictions = model1.predict(features_test_oe)
rmse = mean_squared_error(target_test, predictions)**0.5
print(rmse)

1940.5570637149538
Wall time: 15.5 ms


In [41]:
%%time
model2 = RandomForestRegressor(random_state=12345, max_depth=4, n_estimators=15)
model2.fit(features_train_oe, target_train)

Wall time: 1.65 s


RandomForestRegressor(max_depth=4, n_estimators=15, random_state=12345)

In [42]:
%%time
predictions = model2.predict(features_test_oe)
rmse = mean_squared_error(target_test, predictions)**0.5
print(rmse)

2577.138262942656
Wall time: 39.9 ms


In [43]:
%%time
model3 = lgb.LGBMRegressor(random_state=12345, n_estimators=500,num_leaves=400)
model3.fit(features_train_oe, target_train)

Wall time: 11.4 s


LGBMRegressor(n_estimators=500, num_leaves=400, random_state=12345)

In [44]:
%%time
predictions = model3.predict(features_test_oe)
rmse = mean_squared_error(target_test, predictions)**0.5
print(rmse)

1620.5885283540615
Wall time: 1.31 s


Модель решающего дерева показала наилучший результат, затем следует модель LGBMRegressor, а на третьем месте находится модель RandomForestRegressor.

## Финальны вывод

В процессе работы была выполнена предобработка данных, в ходе которой были обнаружены аномалии, дубликаты и пропущенные значения. Аномалии и ненужные столбцы были исключены, а дубликаты и пропущенные значения были удалены.

Были рассмотрены три модели: случайный лес, решающее дерево и LGBMRegressor. Проведен был подбор оптимальных параметров и оценка качества моделей на основе метрики RMSE. Лучшей моделью с RMSE значением 1576.881736930125 оказалась LGBMRegressor.

При анализе времени обучения моделей на тренировочной выборке было выяснено, что модели следуют следующему порядку: решающее дерево, LGBMRegressor, случайный лес.

Мы рекомендуем использовать модель LGBMRegressor для заказчика, так как она обладает оптимальными характеристиками как по метрике RMSE, так и по скорости обучения и предсказания.